In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Conditional Probability Review

**Eye Color Problem**

The color of a person's eyes is determined by a single pair of
genes.  If they are both blue-eyed genes, then the person will have
blue eyes; if either is a brown-eyed gene, then the person will have
brown eyes.  (The brown-eyed gene is said to be *dominant* over
the blue-eyed one.)  

A newborn child independently receives one
eye-color gene from each of its parents, and the gene it receives
from a parent is equally likely to be either of the two eye-color
genes that the parent carries.

Suppose that Smith and both of his
parent have brown eyes, but Smith's sister has blue eyes.
Furthermore, suppose that Smith's wife has blue eyes.

1. What is the probability that Smith possesses a blue-eyed gene?

1.  What is the probability that the Smiths' first child will
  have blue eyes?

1. If their first child has brown eyes, what is the probability that
 their next child will also have brown eyes?

# Exact Permutation Tests

Let's start with a simple example. Consider the average course grades for EEE 5544 over the past 6 years, omitting 2017 (when I didn't teach it):

|Year | Grade|
|-|-|
|2013|74.1|
|2014|74.5|
|2015|79.4|
|2016|79.0|
|2018|78.4|

Let's check the means for the first 2013-2014 vs 2015-2018:

In [6]:
grades1 = np.array([74.1,74.5])
grades2 = np.array([79.4,79.0,78.4])

In [7]:
grades1.mean(), grades2.mean()

(74.3, 78.93333333333334)

In [9]:
diff = grades2.mean()- grades1.mean()
diff

4.63333333333334

Has the average grade increased over time, or could this just be attributable to the small number of samples?

We could perform bootstrap resampling under the null hypothesis, but the data is so small, we might get repeats of the same samples. 

An alternative is to pool the data and try **all** the ways to rsample the data into samples of size 2 and 3:

# Exact Permutation Test

In Fisher's exact permutation test (for binary hypothesis testing), we:

1. Pool the data
2. Find every partition of the data into two subsets (with sizes equal to the original samples)
3. Measure the sample statistics for each new sample
4. Determine whether the new sample statistics are as extreme as the original observation

First, we demonstrate how to find all the subsets of a certain size for a list: 

In [12]:
import itertools

In [13]:
itertools.combinations([1,2,3,4,5,6],3)

In [16]:
samples = list(itertools.combinations([1,2,3,4,5,6],3))
samples

[(1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 2, 6),
 (1, 3, 4),
 (1, 3, 5),
 (1, 3, 6),
 (1, 4, 5),
 (1, 4, 6),
 (1, 5, 6),
 (2, 3, 4),
 (2, 3, 5),
 (2, 3, 6),
 (2, 4, 5),
 (2, 4, 6),
 (2, 5, 6),
 (3, 4, 5),
 (3, 4, 6),
 (3, 5, 6),
 (4, 5, 6)]

In [17]:
len(samples)

20

In [18]:
from scipy.special import binom

In [19]:
binom(6,3)

20.0

Thus, we can pull all the samples of size 2 from our data:

The trick is that for each sample, we need to find the remaining set to go in the other sample.  We can use numpy's ```setxor1d``` method

Now we are ready to conduct our exact permutation test:

There is not enough data to support that the average score has increased in the last 3 years of the study. 

(Since there are only 10 permutations, we can never get a $p$-value less than 0.1 with data sets of 2 values and 3 values)

Unlike resampling, this approach tries **every** way or redistributing the pooled data. So why not always use it?

Consider the previous example, where the data for the 50 states were split into two clusters of size 42 and 8. How many different combinations of samples would be created in the exact permutation test? 

So, how can we handle a case like this?

# Monte Carlo Permutation Test

Let's consider the following question that should be of interest to engineers: Do males score higher on standardized high school math and science tests than females?

We will use data from the "High School & Beyond (HS&B)" survey conducted y the National Center for Education Statistics:

https://nces.ed.gov/surveys/hsb/index.asp

We are using a CSV file with 200 randomly selected observations from that data set. The CSV file is available here:

https://github.com/rpruim/OpenIntro/blob/master/data/hsb2.csv

A brief discussion of the different fields is available at

http://www.philender.com/courses/762/notes1/about_hsb2.html


In [2]:
df=pd.read_csv("hsb2.csv")

In [3]:
df

,id,gender,race,ses,schtyp,prog,read,write,math,science,socst
0,70,male,white,low,public,general,57,52,41,47,57
1,121,female,white,middle,public,vocational,68,59,53,63,61
2,86,male,white,high,public,general,44,33,54,58,31
3,141,male,white,high,public,vocational,63,44,47,53,56
4,172,male,white,middle,public,academic,47,52,57,53,61
5,113,male,white,middle,public,academic,44,52,51,63,61
6,50,male,african american,middle,public,general,50,59,42,53,61
7,11,male,hispanic,middle,public,academic,34,46,45,39,36
8,84,male,white,middle,public,general,63,57,54,58,51
9,48,male,african american,middle,public,academic,57,55,52,50,51


We want to partition this dataframes into two seperate dataframes according to gender. This is easy to do in pandas, but it looks a little strange. First we get a boolean Series the contains True for whichever rows we want to keep:

Now, if we pass that Series as indices to the original dataframe, it will return a new dataframe with only those rows:

Let's start with math scores:

What can you infer from this plot?

Let's consider science scores:

Is this result statistically significant? 

How many different sample combinations are there in the exact permutation test?

Instead, let's sample 10,000 of the permutations. Since there are so many, we will randomly select permutations, so there is some small probability of repeat

To choose the samples, we will permute the pooled data and then subdivide into the appropriate sizes:

What is your conclusion?

In [4]:
# Here are all the average values
tests=["read","write","math","science","socst"]
for test in tests:
    print(test.ljust(8),"--\t", "Males:",males[test].mean(),"\tFemales:",females[test].mean())


NameError: name 'males' is not defined

**Lecture 13 Assignment**

1. Conduct a bootstrap resampling test for statistical significance of the difference in average science scores between the genders. Compare the $p$-value for the bootstrap test vs. that found via the Monte Carlo permutation test.

2. Generate a table of the median values for all tests, by gender.

2. What is the median writing score for men vs women? Using a Monte Carlo permutation test (not bootstrap resampling), answer the following questions. Is the difference statistically significant at the p<0.01 level? What is the 99% confidence interval for the difference in medians?
